In [ ]:
import numpy as np
import xarray as xr
import scipy.stats as st
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import requests
import os,errno
import sys
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import cartopy.feature as cfeature
import datetime as dt
import pandas as pd
import time
import urllib.request
import metpy.calc as mpcalc
import salem
import scipy.optimize as opt
import warnings
import geopy.distance
import matplotlib as mpl
import matplotlib.colors as mcolors
from matplotlib.lines import Line2D
from sklearn_som.som import SOM

from scipy.ndimage.measurements import label
from scipy.ndimage import binary_dilation

warnings.filterwarnings('ignore')

dir_data='/Users/lexihenny/'
dir1='/Volumes/LaCie/'

In [ ]:
yrs=np.arange(1983,2001,1)

dir='/Users/lexihenny/Past_nogauge/Daily/'
dir2='/Volumes/Extreme Pro/'
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    
    days=np.arange(1,366,1)
    if year==1979:
        days=np.arange(2,366,1)
    if year%4==0:
        days=np.arange(1,367,1)
    for j in days:
    #for j in range(1,5,1):
        print(j)
        if j<10:
            day_str='00'+str(j)
        elif 10<=j<=99:
            day_str='0'+str(j)
        else:
            day_str=str(j)
        ds=xr.open_dataset(dir+str(year)+day_str+'.nc')
        ds.transpose('time','lat','lon')
        precip=ds['precipitation']
        
        if j==days[0]:
            precip_concat=precip
        else:
            precip_concat=xr.concat([precip_concat,precip],dim='time')
            
    dk=xr.Dataset()
    dk['precip']=(('time','lat','lon'),precip_concat.values)
    dk.coords['time']=precip_concat.time.values
    dk.coords['lat']=precip_concat.lat.values
    dk.coords['lon']=precip_concat.lon.values
    
    dk['time'].attrs["description"]='Datetime'
    dk['lat'].attrs["units"]='degrees_north'
    dk['lon'].attrs["units"]='degrees_east'
    dk['precip'].attrs["units"]='mm*day-1'
    
    try:
        os.remove(dir2+'mswep_past_nogauge_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir2+'mswep_past_nogauge_'+str(year)+'.nc',mode='w',format='NETCDF4')
    dk.close()

In [ ]:
yrs=np.arange(1979,1980,1)
for i in range(len(yrs)):
    year=yrs[i]
    print(year)
    ds=xr.open_dataset(dir+'mswep_past_'+str(year)+'.nc')
    precip=ds['precip']
    
    lons_west=[x for x in precip.lon.values if x<-90]
    lons_east=[x for x in precip.lon.values if x>=90]
    precip_west=precip.sel(lon=lons_west)
    precip_east=precip.sel(lon=lons_east)
    precip_west['lon']=[x+360. for x in lons_west]
    precip_east['lon']=[x-360. for x in lons_east]
    precip=xr.concat([precip_east,precip,precip_west],dim='lon')
    
    precip=precip.interp(lat=np.arange(-90,90,0.25),lon=np.arange(-270,270,0.25))
    lons_center=[x for x in precip.lon.values if -180<=x<180]
    precip=precip.sel(lon=lons_center)
    
    dk=xr.Dataset()
    dk['precip']=(('time','lat','lon'),precip.values)
    dk.coords['time']=precip.time
    dk.coords['lat']=precip.lat
    dk.coords['lon']=precip.lon
    try:
        os.remove(dir_data+'mswep_past_deres_'+str(year)+'.nc')
    except OSError:
        pass
    dk.to_netcdf(dir_data+'mswep_past_deres_'+str(year)+'.nc',mode='w',format='NETCDF4')
    ds.close()
    precip.close()
    dk.close()